# Manifold GP Semi-Supervised Learning via Precision Matrix on 1D Manifold

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import torch
import gpytorch
import numpy as np
import os
import scipy.spatial as ss

from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from gpytorch.priors import NormalPrior, GammaPrior

## Dataset Preprocessing

### Load & Settings

In [2]:
dataset = 'protein' # ['protein','elevators', 'ctslice']
cut = 10000

if dataset == 'protein':
    data = np.loadtxt('datasets/protein.csv', delimiter=",")[:cut]
    sampled_x, sampled_y = data[:, 1:], data[:, 0]
elif dataset == 'elevators':
    data = np.array(loadmat('datasets/elevators.mat')['data'])
    sampled_x, sampled_y = data[:, :-1], data[:, -1]
elif dataset == 'ctslice':
    data = np.loadtxt('datasets/ctslice.csv', delimiter=",")[:cut]
    sampled_x, sampled_y = data[:, :-1], data[:, -1]
    
# remove coincident points
sampled_x, id_unique = np.unique(sampled_x, axis=0, return_index=True)
sampled_y = sampled_y[id_unique]

# cut between 0.01 and 0.99 quantile of distances
kd_tree = ss.KDTree(sampled_x)
v = kd_tree.query(sampled_x, k=2)[0][:, 1]
idx = np.argsort(v)
percentile_start = int(np.round(idx.shape[0]*0.10))
percentile_end = int(np.round(idx.shape[0]*0.90))
sampled_x = sampled_x[idx[percentile_start:percentile_end], :]
sampled_y = sampled_y[idx[percentile_start:percentile_end]]
m = sampled_x.shape[0]

normalize_features = True
normalize_labels = True

### Trainset & Testset

In [3]:
split = int(0.2 * m)

train_x, train_y = sampled_x[:split], sampled_y[:split]
test_x, test_y = sampled_x[split:], sampled_y[split:]

train_idx = torch.arange(0, split)
sampled_x = torch.from_numpy(sampled_x).float()
train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_y).float()
test_x = torch.from_numpy(test_x).float()
test_y = torch.from_numpy(test_y).float()

if normalize_features:
    mu_x, std_x = sampled_x.mean(dim=-2, keepdim=True), sampled_x.std(dim=-2, keepdim=True) + 1e-6
    sampled_x.sub_(mu_x).div_(std_x)
    train_x.sub_(mu_x).div_(std_x)
    test_x.sub_(mu_x).div_(std_x)
    
if normalize_labels:
    mu_y, std_y = train_y.mean(), train_y.std()
    train_y.sub_(mu_y).div_(std_y)
    test_y.sub_(mu_y).div_(std_y)

### Hyperparameters Priors

In [4]:
import scipy.spatial as ss
neighbors = 10
kd_tree = ss.KDTree(sampled_x)
v = np.sort(kd_tree.query(sampled_x, k=neighbors+1)[0][:, 1:].ravel())
percentile_99 = int(np.round(v.shape[0]*0.99))
gamma_rate = 100.0/np.std(v)
gamma_concentration = gamma_rate * v[percentile_99] + 1

### Move Data to Device

In [5]:
sampled_x = sampled_x.contiguous()
# train_idx = train_idx.contiguous()
train_x, train_y = train_x.contiguous(), train_y.contiguous()
test_x, test_y = test_x.contiguous(), test_y.contiguous()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
sampled_x = sampled_x.to(device)
# train_idx = train_idx.to(device)
train_x, train_y = train_x.to(device), train_y.to(device)
test_x, test_y = test_x.to(device), test_y.to(device)

## Model

In [6]:
%%capture
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
    noise_prior=None  # NormalPrior(torch.tensor([0.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

kernel = gpytorch.kernels.ScaleKernel(
    RiemannMaternKernel(
        nu=3,
        nodes=sampled_x,
        neighbors=50,
        operator="randomwalk",
        modes=200,
        ball_scale=3.0,
        support_kernel=gpytorch.kernels.RBFKernel(),
        epsilon_prior=GammaPrior(gamma_concentration, gamma_rate),
        lengthscale_prior=None, # InverseGammaPrior(igamma_concentration, igamma_rate)
    ),
    outputscale_prior=None  # NormalPrior(torch.tensor([1.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

model = RiemannGP(train_x, train_y, likelihood, kernel, train_idx).to(device)

## Train

In [7]:
%%capture
hypers = {
    'likelihood.noise_covar.noise': 1e-2, # 0.037,
    'covar_module.base_kernel.epsilon': 0.5, # 0.029,
    'covar_module.base_kernel.lengthscale': 0.5, # 5.130,
    'covar_module.outputscale': 1.0, # 0.0656,
    'covar_module.base_kernel.support_kernel.lengthscale': 1.0,
}
model.initialize(**hypers)

In [8]:
model.manifold_informed_train(lr=1e-2, iter=100, norm_step_size=100, verbose=True)

Iter: 0, LR: 0.010, Loss: 963.283, NoiseVar: 0.010, SignalVar: 714901.812, Lengthscale: 0.500, Epsilon: 0.500
Iter: 1, LR: 0.010, Loss: 952.777, NoiseVar: 0.010, SignalVar: 714901.812, Lengthscale: 0.504, Epsilon: 0.504
Iter: 2, LR: 0.010, Loss: 947.386, NoiseVar: 0.010, SignalVar: 714901.812, Lengthscale: 0.508, Epsilon: 0.508
Iter: 3, LR: 0.010, Loss: 941.575, NoiseVar: 0.010, SignalVar: 714901.812, Lengthscale: 0.512, Epsilon: 0.512
Iter: 4, LR: 0.010, Loss: 938.701, NoiseVar: 0.010, SignalVar: 714901.812, Lengthscale: 0.515, Epsilon: 0.516
Iter: 5, LR: 0.010, Loss: 934.626, NoiseVar: 0.010, SignalVar: 714901.812, Lengthscale: 0.519, Epsilon: 0.520
Iter: 6, LR: 0.010, Loss: 929.499, NoiseVar: 0.011, SignalVar: 714901.812, Lengthscale: 0.522, Epsilon: 0.524
Iter: 7, LR: 0.010, Loss: 928.443, NoiseVar: 0.011, SignalVar: 714901.812, Lengthscale: 0.525, Epsilon: 0.528
Iter: 8, LR: 0.010, Loss: 931.701, NoiseVar: 0.011, SignalVar: 714901.812, Lengthscale: 0.527, Epsilon: 0.532
Iter: 9, L

Iter: 74, LR: 0.010, Loss: 826.052, NoiseVar: 0.009, SignalVar: 714901.812, Lengthscale: 0.520, Epsilon: 0.822
Iter: 75, LR: 0.010, Loss: 826.157, NoiseVar: 0.009, SignalVar: 714901.812, Lengthscale: 0.520, Epsilon: 0.826
Iter: 76, LR: 0.010, Loss: 825.247, NoiseVar: 0.009, SignalVar: 714901.812, Lengthscale: 0.520, Epsilon: 0.831
Iter: 77, LR: 0.010, Loss: 823.781, NoiseVar: 0.009, SignalVar: 714901.812, Lengthscale: 0.520, Epsilon: 0.835
Iter: 78, LR: 0.010, Loss: 825.759, NoiseVar: 0.009, SignalVar: 714901.812, Lengthscale: 0.520, Epsilon: 0.839
Iter: 79, LR: 0.010, Loss: 823.637, NoiseVar: 0.009, SignalVar: 714901.812, Lengthscale: 0.520, Epsilon: 0.843
Iter: 80, LR: 0.010, Loss: 820.813, NoiseVar: 0.009, SignalVar: 714901.812, Lengthscale: 0.520, Epsilon: 0.848
Iter: 81, LR: 0.010, Loss: 820.960, NoiseVar: 0.009, SignalVar: 714901.812, Lengthscale: 0.520, Epsilon: 0.852
Iter: 82, LR: 0.010, Loss: 819.997, NoiseVar: 0.009, SignalVar: 714901.812, Lengthscale: 0.520, Epsilon: 0.856
I

In [9]:
model.manifold_informed_train(lr=1e-2, iter=100, norm_step_size=100, verbose=True)

Iter: 0, LR: 0.010, Loss: 810.431, NoiseVar: 0.008, SignalVar: 652789.250, Lengthscale: 0.520, Epsilon: 0.929
Iter: 1, LR: 0.010, Loss: 808.000, NoiseVar: 0.008, SignalVar: 652789.250, Lengthscale: 0.524, Epsilon: 0.935
Iter: 2, LR: 0.010, Loss: 805.840, NoiseVar: 0.008, SignalVar: 652789.250, Lengthscale: 0.528, Epsilon: 0.941
Iter: 3, LR: 0.010, Loss: 805.908, NoiseVar: 0.008, SignalVar: 652789.250, Lengthscale: 0.531, Epsilon: 0.947
Iter: 4, LR: 0.010, Loss: 803.328, NoiseVar: 0.008, SignalVar: 652789.250, Lengthscale: 0.533, Epsilon: 0.953
Iter: 5, LR: 0.010, Loss: 803.615, NoiseVar: 0.008, SignalVar: 652789.250, Lengthscale: 0.533, Epsilon: 0.959
Iter: 6, LR: 0.010, Loss: 803.678, NoiseVar: 0.008, SignalVar: 652789.250, Lengthscale: 0.533, Epsilon: 0.965
Iter: 7, LR: 0.010, Loss: 802.784, NoiseVar: 0.008, SignalVar: 652789.250, Lengthscale: 0.531, Epsilon: 0.971
Iter: 8, LR: 0.010, Loss: 801.476, NoiseVar: 0.008, SignalVar: 652789.250, Lengthscale: 0.529, Epsilon: 0.977
Iter: 9, L

Iter: 74, LR: 0.010, Loss: 783.522, NoiseVar: 0.007, SignalVar: 652789.250, Lengthscale: 0.529, Epsilon: 1.268
Iter: 75, LR: 0.010, Loss: 783.757, NoiseVar: 0.007, SignalVar: 652789.250, Lengthscale: 0.528, Epsilon: 1.269
Iter: 76, LR: 0.010, Loss: 784.309, NoiseVar: 0.007, SignalVar: 652789.250, Lengthscale: 0.528, Epsilon: 1.271
Iter: 77, LR: 0.010, Loss: 784.165, NoiseVar: 0.007, SignalVar: 652789.250, Lengthscale: 0.527, Epsilon: 1.272
Iter: 78, LR: 0.010, Loss: 783.647, NoiseVar: 0.007, SignalVar: 652789.250, Lengthscale: 0.527, Epsilon: 1.274
Iter: 79, LR: 0.010, Loss: 784.256, NoiseVar: 0.007, SignalVar: 652789.250, Lengthscale: 0.527, Epsilon: 1.275
Iter: 80, LR: 0.010, Loss: 783.588, NoiseVar: 0.007, SignalVar: 652789.250, Lengthscale: 0.527, Epsilon: 1.277
Iter: 81, LR: 0.010, Loss: 783.286, NoiseVar: 0.007, SignalVar: 652789.250, Lengthscale: 0.528, Epsilon: 1.278
Iter: 82, LR: 0.010, Loss: 784.336, NoiseVar: 0.007, SignalVar: 652789.250, Lengthscale: 0.528, Epsilon: 1.279
I

## Evaluation

In [10]:
%%capture
likelihood.eval()
model.eval()


## Metrics

In [11]:
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds_test = likelihood(model(test_x))

    mean_test = preds_test.mean
        
    error = test_y - preds_test.mean
    covar = preds_test.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    
    rmse = (error.square().sum()/test_y.shape[0]).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/test_y.shape[0]
    
print("RMSE: ", rmse)
print("NLL: ", nll)

RMSE:  tensor(1.1336, device='cuda:0')
NLL:  tensor(43.3277, device='cuda:0')
